 # Navigation

 ---

 In this notebook, you can find all the necessary code for training an agent with an Double DQN Network to learn how to collect yellow and keep away from blue bananas in an [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) environment.

 ### 1. Import all necessary packages

In [ ]:


from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import torch
from ddqn_agent import Agent


### 2. Open Environment and check it out
We use the UnityEnvironment `Banana` for Windows 64-bit so make sure you run this notebook on a Windows machine.
Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:

env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]


# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)


 ### 2. Examine the State and Action Spaces

 The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
 - `0` - walk forward
 - `1` - walk backward
 - `2` - turn left
 - `3` - turn right

 The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana.

 ### 3. Train the agent in the environment

In the next step we define our Double-DQN Agent and initialize it with the proper state and action sizes, as well with a random seed.

In [ ]:

agent = Agent(state_size=state_size, action_size=action_size, seed=1337)



Let's define a Deep Q-Learning function to iterate over all episodes and timesteps within that. This function returns a list of the scores of all episodes.
Our target score is an average of 13 over the last 100 episodes. But we are optimistic with our Double DQN approach and trry to reach a average score of 16.

In [ ]:


def dqn(target_score=13 ,n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        state = env_info.vector_observations[0]            # get the current state  
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps).astype(np.int32)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=target_score:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores



So now lets run the function to train and save a plot of the score progression. If the agent reaches our target score, it gets automatically saved as a `checkpoint.pth` file.

In [ ]:
scores = dqn(target_score=16)

Nice after 500 episodes we already reached our average score of 13.
But more impressively we achieved the score of 16 after 849 episodes.

In [ ]:
env.close()
# plot the scores
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

 ### 3. See the trained agent in action
 Lets open up the environment again and see our agent in action. For that we recreate a agent object and load our `checkpoint.pth` file to validate it is working properly.

In [ ]:
env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")

brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=False)[brain_name] # reset the environment

action_size = brain.vector_action_space_size
state = env_info.vector_observations[0]
state_size = len(state)

agent = Agent(state_size=state_size, action_size=action_size, seed=1337)
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
counter=0
while True:
    action = agent.act(state).astype(np.int32)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    counter+=1
    if done:                                       # exit loop if episode finished
        break

env.close()
print("Score: {}".format(score))
print(counter)